In [1]:
import sys
sys.path.append('..')

In [6]:
from pathlib import Path
from itertools import islice
import csv

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score

import mmr.ds_loading
import mmr.vectorization
import cubert_wrapper

In [62]:
data_root = Path('/home/maxkvant/data/mmr/')

test_classes_vecs_path = data_root / 'mmr_vecs_np'
test_methods_vecs_path = data_root / 'mmr_vecs_wm'
test_ds = mmr.ds_loading.MMRDataset(data_root / 'MoveMethodDataset', test_methods_vecs_path, test_classes_vecs_path, 
                                    torch.FloatTensor, torch.FloatTensor, True, True, precalculated=True)

train_classes_vecs_path = data_root / 'mmr_tr_dsv'
train_methods_vecs_path = data_root / 'mmr_tr_dsvwm_v2'
train_ds = mmr.ds_loading.MMRDataset(data_root / 'mmr_tr_ds', train_methods_vecs_path, train_classes_vecs_path, 
                                     torch.FloatTensor, torch.FloatTensor, True, True, precalculated=True)

In [63]:
c_tr = {i[0] for i in train_ds}
c_te = {i[0] for i in test_ds}
tr_exclude_projects = c_tr & c_te
tr_exclude_projects

{'actor-platform',
 'atlas',
 'buck',
 'crate',
 'deeplearning4j',
 'drools',
 'hbase',
 'hive',
 'jenkins',
 'jstorm',
 'pinpoint',
 'pmd'}

In [7]:
proj_train, proj_val = train_test_split(list(c_tr - tr_exclude_projects))

In [66]:
def ds_to_xy(ds, val_projects, exclude_projects = {}):
    x_train, x_val, y_train, y_val = [], [], [], []
    mn_train, mn_val = [], []
    for project, mn, _, mv, cv, tgt in ds:
        if project in val_projects:
            x_val.append((mv, cv))
            y_val.append(tgt)
            mn_val.append(mn)
        elif project not in exclude_projects:
            x_train.append((mv, cv))
            y_train.append(tgt)
            mn_train.append(mn)
    return x_train, x_val, y_train, y_val, mn_train, mn_val


x_train, x_val, y_train, y_val, mn_train, mn_val = ds_to_xy(train_ds, proj_val, tr_exclude_projects)
x_test, _, y_test, _, mn_test, _ = ds_to_xy(test_ds, {})

In [67]:
dl_train = torch.utils.data.DataLoader(list(zip(x_train, y_train)), batch_size=1)
dl_val = torch.utils.data.DataLoader(list(zip(x_val, y_val)), batch_size=1)

In [12]:
class RNNClassifier(nn.Module):
    def __init__(self, hidden_dim, n_channels):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_channels = n_channels
        self.method_rnn = nn.LSTM(1024, hidden_dim, n_channels, batch_first=True)
        self.class_rnn = nn.LSTM(1024, hidden_dim, n_channels, batch_first=True)
        self.linear_clf = nn.Sequential(
            nn.Linear(hidden_dim * n_channels * 2, 512),
            nn.PReLU(),
            nn.Linear(512, 512),
            nn.PReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        method_line_embs, class_line_embs = x
        _, (_, method_last_c) = self.method_rnn(method_line_embs)
        _, (_, class_last_c) = self.class_rnn(class_line_embs)
        rnn_cs = torch.cat((method_last_c.reshape((-1, self.hidden_dim * self.n_channels)), 
                            class_last_c.reshape((-1, self.hidden_dim * self.n_channels))), axis=1)
        return self.linear_clf(rnn_cs)

In [68]:
class CNNClassifier(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.method_convs = nn.Sequential(
            nn.Conv1d(1024, 1024, 5, padding=2),
            nn.Tanh(),
            nn.Conv1d(1024, 1024, 5, padding=2),
            nn.Tanh()
        )
        self.class_convs = nn.Sequential(
            nn.Conv1d(1024, 1024, 5, padding=2),
            nn.Tanh(),
            nn.Conv1d(1024, 1024, 5, padding=2),
            nn.Tanh(),
        )
        self.linear_clf = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.PReLU(),
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 512),
            nn.PReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        method_line_embs, class_line_embs = x
        method_line_embs, class_line_embs = method_line_embs.permute(0, 2, 1), class_line_embs.permute(0, 2, 1)
        conved = torch.cat((self.method_convs(method_line_embs).mean(axis=2), 
                            self.class_convs(class_line_embs).mean(axis=2)),
                           axis=1)
        return self.linear_clf(conved)

In [14]:
class AttnClassifier(nn.Module):
    def __init__(self, ):
        super().__init__()
        self.method_convs = nn.Sequential(
            nn.Conv1d(1024, 1, 1)
        )
        self.class_convs = nn.Sequential(
            nn.Conv1d(1024, 1, 1)
        )
        self.linear_clf = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.PReLU(),
            nn.Linear(1024, 512),
            nn.PReLU(),
            nn.Linear(512, 512),
            nn.PReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        method_line_embs, class_line_embs = x
        method_line_embs, class_line_embs = method_line_embs.permute(0, 2, 1), class_line_embs.permute(0, 2, 1)
        method_weights = torch.softmax(self.method_convs(method_line_embs), axis=2)
        class_weights = torch.softmax(self.class_convs(class_line_embs), axis=2)
        method_rep = torch.sum(method_weights * method_line_embs, axis=2)
        class_rep = torch.sum(class_weights * class_line_embs, axis=2)
        reps = torch.cat((method_rep, class_rep), axis=1)
        return self.linear_clf(reps)

In [20]:
def train(model, n_epochs, dl_train, dl_val, lr, wd, save_path):
    loss_function = nn.BCELoss()
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    best_score = -1
    
    for epoch in range(n_epochs):
        train_losses, val_losses, val_tgts, val_preds = [], [], [], []
        for x, y in tqdm(dl_train):
            x = tuple(i.cuda() for i in x)
            y = y.cuda()
            opt.zero_grad()
            pred = model(x)
            y = y.unsqueeze(1).float()
            loss = loss_function(pred, y)
            loss.backward()
            opt.step()
            train_losses.append(loss.detach())

        with torch.no_grad():
            for x, y in tqdm(dl_val):
                x = tuple(i.cuda() for i in x)
                pred = model(x).cpu()
                y = y.unsqueeze(1).float()
                loss = loss_function(pred, y)
                val_losses.append(loss)
                val_tgts.append(y[0, 0])
                val_preds.append(pred[0, 0])
        train_loss = torch.tensor(train_losses).mean()
        val_loss = torch.tensor(val_losses).mean()
        val_tgts, val_preds = torch.tensor(val_tgts), torch.tensor(val_preds)
        print((val_preds > .5).float().mean(), val_tgts.float().mean())
        val_f1 = f1_score(val_tgts, val_preds > .5)
        val_roc = roc_auc_score(val_tgts, val_preds)
        if val_f1 > best_score:
            best_score = val_f1
            torch.save(model.state_dict(), save_path)
        print(f'Epoch {epoch} loss: train {train_loss:.4f} val {val_loss:.4f} f1: {val_f1:.4f}, ROC AUC: {val_roc:.4f}')

In [ ]:
rnn_model = RNNClassifier(1024, 2).cuda()
train(rnn_model, 20, dl_train, dl_val, 1e-4, 1e-5)

In [74]:
cnn_model = CNNClassifier().cuda()
train(cnn_model, 40, dl_train, dl_val, 1e-5, 1e-6, 'seq_cnn_wm.pt')

100%|██████████| 7459/7459 [00:22<00:00, 330.90it/s]


tensor(0.2465) tensor(0.5446)


  0%|          | 10/23679 [00:00<04:02, 97.41it/s]

Epoch 0 loss: train 0.4471 val 0.7125 f1: 0.5785, ROC AUC: 0.8342


100%|██████████| 7459/7459 [00:22<00:00, 330.29it/s]


...

tensor(0.4985) tensor(0.5446)


  0%|          | 11/23679 [00:00<03:49, 102.98it/s]

Epoch 38 loss: train 0.1367 val 0.7493 f1: 0.8288, ROC AUC: 0.8916


100%|██████████| 7459/7459 [00:22<00:00, 334.82it/s]


tensor(0.5021) tensor(0.5446)
Epoch 39 loss: train 0.1317 val 0.7183 f1: 0.8305, ROC AUC: 0.8915


In [ ]:
attn_model = AttnClassifier().cuda()
train(attn_model, 20, dl_train, dl_val, 1e-4, 1e-4)

In [75]:
model = CNNClassifier()
model.load_state_dict(torch.load('seq_cnn_wm.pt'))
model.train(False)
with torch.no_grad():
    pred = np.array([model(i) for i in torch.utils.data.DataLoader(x_test)])
    print(roc_auc_score(y_test, pred), f1_score(y_test, pred > .5))

0.882351462971083 0.8094045095394105


In [35]:
from collections import defaultdict


def per_project_f1(mns, tgts, preds, projs):
    m_true_scores = defaultdict(dict)
    m_false_scores = defaultdict(lambda: defaultdict(list))
    for mn, tgt, pred, proj in zip(mns, tgts, preds, projs):
        if tgt:
            m_true_scores[proj][mn] = pred
        else:
            m_false_scores[proj][mn].append(pred)
    f1s = []
    for proj in m_true_scores.keys():
        tp, n_refs, n_methods = 0, 0, 0
        for mn, mts in m_true_scores[proj].items():
            if not m_false_scores[proj][mn]:
                continue
            if max(mts, *m_false_scores[proj][mn]) > .5:
                n_refs += 1
                if mts > max(m_false_scores[proj][mn]):
                    tp += 1
            n_methods += 1
        precision = tp / n_refs
        recall = tp / n_methods
        f1s.append(2 * precision * recall / (precision + recall))
    return np.mean(f1s)

In [37]:
proj_test = [name for (name, *_) in test_ds]

In [76]:
per_project_f1(mn_test, y_test, pred, proj_test)

0.8165950197672277